In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

tokenizer = AutoTokenizer.from_pretrained("kreimben/CodeMind-gemma", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("kreimben/CodeMind-gemma")

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--kreimben--CodeMind-gemma. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [2]:
model = model.to('cuda')

In [9]:
%%time

res = get_completion(
    'i dont know about word break problem. please give me a approch about this problem.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


CPU times: total: 1min 5s
Wall time: 2min 14s


In [10]:
answer = res.split('model')[-1]

In [11]:
from IPython.display import display, Markdown

answer = answer.replace('\\\\', '\\')

display(Markdown(answer))

 Answer: Who do you mean, I\'m bored</start_of_turn>
<end_of_turn></end_of_turn>
</question>
</start_of_turn>
<start_of_turn>question question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question? question?</start_of_turn>
<start_of_turn>answer This is not a question</answer>
<start_of_turn>question Who\'s your favorite charcter in a show?</start_of_turn>
<start_of_turn>question Who\'s your favorite charcter in a show?</start_of_turn>
Question of the day! Who\'s your favorite charcter in a show?</start_of_turn>
Question of the day! Who\'s your favorite charcter in a show?</start_of_turn>
Question of the day! Who\'s your favorite charcter in a show?</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food??</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food?</start_of_turn>
question How do people get their food?</start_of_turn>
question What are you doing?</start_of_turn>
question What are you doing?</start_of_turn>
question What are you doing?</start_of_turn>
question What are you doing?</start_of_turn>
question Who\'s your favorite charcter in a show?</start_of_turn>
question What are you doing?</start_of_turn>
question What are you doing?</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood?</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn>question Howdopeoplegetfood??</start_of_turn></start_of_turn>howdopeoplegetfood?</question>howdopeoplegetfood?</question></start_of_turn>question Howdopeoplegetfood?</question></start_of_turn>question Howdopeoplegetfood?</question>question Howdopeopleget